Наши данные

In [9]:
import pandas as pd
import re

pth = '/home/sondors/Documents/price/BERT_NER/csv/Kazakov_241123/Michelin_character.xlsx'   
df = pd.read_excel(pth, dtype=str)
df
# print(df.keys())

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Комментарии,Наименование,Модель,Ширина шины,Высота профиля,...,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35
0,99353,NaN,8101,NaN,7313,NaN,Michelin Cross Climate 175/65 R14 86H XL,CrossClimate,175,65,...,NaN,NaN,Auto,ШИНЫ :: Michelin,Michelin,http://pitstop33.ru/img/tire/tumb/1453431993.jpg,[8101],2023-05-26 20:23:08,только 1 шт. Шиномонтаж -25%,1731212315
1,99353,NaN,8101,NaN,7313,NaN,Michelin Cross Climate 185/60 R14 86H XL,CrossClimate,185,60,...,NaN,NaN,Auto,ШИНЫ :: Michelin,Michelin,http://pitstop33.ru/img/tire/tumb/1453431993.jpg,[8101],2023-05-26 20:23:08,только 1 шт. Шиномонтаж -25%,1729358536
2,99353,NaN,8101,NaN,7313,NaN,Michelin Cross Climate 185/60 R15 88V XL,CrossClimate,185,60,...,NaN,NaN,Auto,ШИНЫ :: Michelin,Michelin,http://pitstop33.ru/img/tire/tumb/1453431993.jpg,[8101],2023-05-26 20:23:08,только 3 шт. Шиномонтаж -25%,1732091573
3,73619,NaN,8101,NaN,7313,NaN,Michelin Cross Climate 185/65 R14 90H,CrossClimate,185,65,...,NaN,NaN,Auto,ШИНЫ :: Michelin,Michelin,http://33market.ru/img/tire/tumb/1453431993.jpg,"[8101, 1403]",2023-01-19 00:23:03,"Продажа от 1 шт., Шиномонтаж - 25%.",1657276894
4,89908,NaN,8101,NaN,7313,NaN,Michelin Cross Climate 185/65 R15 92V,CrossClimate,185,65,...,NaN,NaN,Auto,Шины,Michelin,http://garagewheels.ru/upload/iblock/a35/3u8ov...,"[8101, 829805]",2023-04-03 13:10:28,Мин. кол-во 2 шт. Скидка 15% на шиномонтаж,1689751384
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23428,99353,NaN,8101,NaN,7313,NaN,"Michelin XTE2+ 235/75 R17,5 143/141J",XTE 2 Plus,235,75,...,NaN,NaN,Auto,ШИНЫ :: Michelin,Michelin,http://pitstop33.ru/img/tire/tumb/1453774399.jpg,[8101],2023-05-26 20:23:08,только 1 шт. Шиномонтаж -25%,1729379635
23429,93135,NaN,8101,NaN,7313,NaN,Michelin XZA2 Energy 295/60 R22.5 150/147K Рул...,XZA2 Energy,295,60,...,299752,NaN,Auto,Автотовары :: Шины и диски :: Шины :: Летние ш...,MICHELIN,https://main-cdn.sbermegamarket.ru/hlr-system/...,[8101],2023-06-15 23:41:54,NaN,1742147129
23430,98892,NaN,8101,NaN,7313,NaN,Michelin XZA2 Energy 295/60 R22.5 150K,XZA2 Energy,295,60,...,NaN,Active,Auto,Michelin :: XZA2 Energy,Michelin,NaN,"[8101, 8119, 8104, 2005]",2023-08-21 05:35:12,NaN,1776685698
23431,61893,NaN,8101,NaN,7313,NaN,Michelin XZM 225/75 R15 149A5,XZM,225,75,...,110079,NaN,Auto,Товары для авто :: Летние,Michelin,https://www.bs-tyres.ru/img/image/%D0%B8%D0%B7...,[8101],2023-07-24 14:40:39,NaN,1660836863


In [10]:
print(df.keys())

Index(['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4',
       'Комментарии', 'Наименование', 'Модель', 'Ширина шины',
       'Высота профиля', 'Диаметр', 'Индекс нагрузки и скорости', 'XL',
       'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16',
       'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20',
       'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24',
       'Unnamed: 25', 'Unnamed: 26', 'Unnamed: 27', 'Unnamed: 28',
       'Unnamed: 29', 'Unnamed: 30', 'Unnamed: 31', 'Unnamed: 32',
       'Unnamed: 33', 'Unnamed: 34', 'Unnamed: 35'],
      dtype='object')


In [11]:
def process_text(input_text):
    def separate_letters_and_numbers(input_text):
        # Используем регулярное выражение для поиска сочетаний букв и цифр
        pattern = re.compile(r'(\D+|\d+)')
        
        # Используем findall для нахождения всех сочетаний
        matches = pattern.findall(input_text)
        # Возвращаем строку с пробелами между буквами и цифрами
        return ' '.join(matches)

    processed_text = separate_letters_and_numbers(input_text)    
    processed_text = processed_text.replace("|", " | ")
    processed_text = processed_text.replace("(", " ( ")
    processed_text = processed_text.replace(")", " ) ")
    processed_text = processed_text.replace("[", " [ ")
    processed_text = processed_text.replace("]", " ] ")
    # Убираем повторяющиеся пробелы
    processed_text = re.sub(r'  +', ' ', processed_text)
    return processed_text

def create_bio_tags(row):
    def remove_word(line, word):
        # Преобразуем обе строки в нижний регистр для сравнения без учета регистра
        lower_line = line.lower()
        lower_word = word.lower()

        # Проверяем, содержится ли подстрока в строке
        if lower_word in lower_line:
            # Используем регулярное выражение для удаления подстроки (с учетом регистра)
            line = re.sub(fr'\b{re.escape(word)}\b', '', line, flags=re.IGNORECASE).strip()

        return line
        
    name = str(row['offer'])
    width = str(row['width'])
    height = str(row['height'])
    radius = str(row['radius'])
    brand = row['brand']
    line = str(row['line'])
    v_ind = str(row['v_ind'])

    # Создаем пустой список для хранения тегов
    tags = ['O'] * len(name.split())
    Begin_line = True
    Begin_v_ind = True
    Begin_radius = True
    # Проверяем каждое слово в наименовании и устанавливаем соответствующий тег
    for i, word in enumerate(name.split()):
        if word == '/':
            continue
        
        #   width
        if word == width:
            tags[i] = 'B-width'
            width = width.replace(word,"")

        #   height
        elif word == height:
            tags[i] = 'B-height'
            height = height.replace(word,"")

        #   radius    
        elif word.upper() in radius.upper():
            if radius.replace(" ","") == "":
                continue

            if Begin_radius:
                tags[i] = 'B-radius'
                Begin_radius = False
            else:
                tags[i] = 'I-radius'
            radius = radius.replace(word.lower(),"").replace(word.upper(),"")

        #   brand
        elif word in brand:
            tags[i] = 'B-brand'

        #   v_ind
        elif word.upper() in v_ind.upper():
            if v_ind.replace(" ","") == "":
                continue

            if Begin_v_ind:
                tags[i] = 'B-v_ind'
                Begin_v_ind = False
            else:
                tags[i] = 'I-v_ind'  
            v_ind = v_ind.replace(word.lower(),"").replace(word.upper(),"")  

        #   line
        elif word.upper() in line.upper() or re.sub(r"[^a-zA-Zа-яА-Я]","", word.upper().replace("СПОРТ", "SPORT")) in line.upper():
            if len(re.sub(r"[^a-zA-Zа-яА-Я]","", word.upper().replace("СПОРТ", "SPORT")))<3 and Begin_line:
                continue
            if line.replace(" ","") == "":
                continue
            
            if Begin_line:
                tags[i] = 'B-line'
                Begin_line = False
            else:
                tags[i] = 'I-line'
            line = min(remove_word(line, word), line.replace(word,""))

    # Собираем теги в строку
    return ','.join(tags)

def change_columns(df):   
    new_df = pd.DataFrame()

    df = df.rename(columns={'Наименование': 'offer'})
    df = df[df['offer'].notna()]

    new_df['offer'] = df['offer']
    new_df['width'] = df['Ширина шины']
    new_df['height'] = df['Высота профиля']
    new_df['radius'] = df['Диаметр']
    new_df['v_ind'] = df['Индекс нагрузки и скорости']
    new_df['brand'] = df['Unnamed: 30']
    new_df['line'] = df['Модель']
    
    def process_column(row, column_name):
        # Проверка, является ли значение в "offer" подстрокой значения из указанной колонки
        contains_column_value = str(row['offer']).find(str(row[column_name])) != -1
        
        # Если значение из колонки не найдено в "offer"
        if not contains_column_value:
            # Замена "." на "," в соответствующей строке из columns_to_process
            row[column_name] = str(row[column_name]).replace('.', ',')
            
            # Повторная проверка
            contains_column_value = str(row['offer']).find(str(row[column_name])) != -1

            # Если проверка не пройдена, удаляем строку из таблицы
            if not contains_column_value:
                return None
        return row
    
    def process_radius1(txt):
        return txt.replace("r", "").replace("R", "")
    
    new_df['radius'] = new_df['radius'].apply(process_radius1)
    columns_to_process = ['width', 'height', 'radius']
    for column in columns_to_process:
        new_df = new_df.apply(process_column, args=(column,), axis=1)

    # Удаляем строки, в которых проверка не пройдена
    new_df = new_df.dropna().reset_index(drop=True)

    def process_radius(txt):
        return "R"+txt
    
    new_df['radius'] = new_df['radius'].apply(process_radius)
    new_df['offer'] = new_df['offer'].apply(process_text)
    new_df['brand'] = new_df['offer'].apply(extract_Michelin)
    new_df['BIO_Tags'] = new_df.apply(create_bio_tags, axis=1)

    keywords = ['width', 'height', 'radius', 'v_ind', 'brand', 'line']

    # Проверяем вхождение каждого ключевого слова в каждую строку
    for keyword in keywords:
        new_df = new_df[new_df['BIO_Tags'].str.contains(keyword, case=False)]

    # Переиндексируем DataFrame после удаления строк
    new_df = new_df.reset_index(drop=True)

    new_df = new_df[new_df['v_ind'].apply(lambda x: len(str(x)) <= 5)]
    # Переиндексация DataFrame после удаления строк
    new_df = new_df.reset_index(drop=True)

    return new_df

def extract_Michelin(text):
    brands = re.findall(r'Michelin|MICHELIN|michelin|МИШЛЕН|Мишлен|мишлен|Мешлен|МЕШЛЕН|МЕШЛЕН|Мишелин|МИШЕЛИН|мишелин|Мишелен|МИШЕЛЕН|мишелин', text)
    return brands

new_df = change_columns(df)
new_df

,offer,width,height,radius,v_ind,brand,line,BIO_Tags
0,Michelin Cross Climate 175 / 65 R 14 86 H XL,175,65,R14,86H,[Michelin],CrossClimate,"B-brand,B-line,I-line,B-width,O,B-height,B-rad..."
1,Michelin Cross Climate 185 / 60 R 14 86 H XL,185,60,R14,86H,[Michelin],CrossClimate,"B-brand,B-line,I-line,B-width,O,B-height,B-rad..."
2,Michelin Cross Climate 185 / 60 R 15 88 V XL,185,60,R15,88V,[Michelin],CrossClimate,"B-brand,B-line,I-line,B-width,O,B-height,B-rad..."
3,Michelin Cross Climate 185 / 65 R 14 90 H,185,65,R14,90H,[Michelin],CrossClimate,"B-brand,B-line,I-line,B-width,O,B-height,B-rad..."
4,Michelin Cross Climate 185 / 65 R 15 92 V,185,65,R15,92V,[Michelin],CrossClimate,"B-brand,B-line,I-line,B-width,O,B-height,B-rad..."
...,...,...,...,...,...,...,...,...
22325,Michelin X-ICE XI 2 225 / 60 R 18 100 T XL,225,60,R18,100T,[Michelin],X-Ice Xi2,"B-brand,B-line,I-line,I-line,B-width,O,B-heigh..."
22326,Michelin XTE 2 245 / 70 R 19 . 5 141 J,245,70,R19.5,141J,[Michelin],XTE 2,"B-brand,B-line,I-line,B-width,O,B-height,B-rad..."
22327,Michelin XZA 2 Energy 295 / 60 R 22 . 5 150 K,295,60,R22.5,150K,[Michelin],XZA2 Energy,"B-brand,B-line,B-radius,I-line,B-width,O,B-hei..."
22328,Michelin XZM 225 / 75 R 15 149 A 5,225,75,R15,149A5,[Michelin],XZM,"B-brand,B-line,B-width,O,B-height,B-radius,I-r..."


In [12]:

# def create_bio_tags(row):
#     def remove_word(line, word):
#         # Преобразуем обе строки в нижний регистр для сравнения без учета регистра
#         lower_line = line.lower()
#         lower_word = word.lower()

#         # Проверяем, содержится ли подстрока в строке
#         if lower_word in lower_line:
#             # Используем регулярное выражение для удаления подстроки (с учетом регистра)
#             line = re.sub(fr'\b{re.escape(word)}\b', '', line, flags=re.IGNORECASE).strip()

#         return line
        
#     name = str(row['offer'])
#     width = str(row['width'])
#     height = str(row['height'])
#     radius = str(row['radius'])
#     brand = row['brand']
#     line = str(row['line'])
#     v_ind = str(row['v_ind'])

#     # Создаем пустой список для хранения тегов
#     tags = ['O'] * len(name.split())
#     Begin_line = True
#     Begin_v_ind = True
#     Begin_radius = True
#     # Проверяем каждое слово в наименовании и устанавливаем соответствующий тег
#     for i, word in enumerate(name.split()):
#         if word == '/':
#             continue
        
#         #   width
#         if word == width:
#             tags[i] = 'B-width'
#             width = width.replace(word,"")

#         #   height
#         elif word == height:
#             tags[i] = 'B-height'
#             height = height.replace(word,"")

#         #   radius    
#         elif word.upper() in radius:
#             if Begin_radius:
#                 tags[i] = 'B-radius'
#                 Begin_radius = False
#             else:
#                 tags[i] = 'I-radius'

#         #   brand
#         elif word in brand:
#             tags[i] = 'B-brand'

#         #   v_ind
#         elif word.upper() in v_ind.upper():
#             if v_ind.replace(" ","") == "":
#                 continue

#             if Begin_v_ind:
#                 tags[i] = 'B-v_ind'
#                 Begin_v_ind = False
#             else:
#                 tags[i] = 'I-v_ind'  
#             v_ind = v_ind.replace(word.lower(),"").replace(word.upper(),"")  

#         #   line
#         elif word.upper() in line.upper() or word.upper().replace("СПОРТ", "SPORT") in line.upper():
#             if len(re.sub(r"[^a-zA-Zа-яА-Я]","", word.upper().replace("СПОРТ", "SPORT")))<3 and Begin_line:
#                 continue
#             if line.replace(" ","") == "":
#                 continue
            
#             if Begin_line:
#                 tags[i] = 'B-line'
#                 Begin_line = False
#             else:
#                 tags[i] = 'I-line'
#             line = min(remove_word(line, word), line.replace(word,""))

#     # Собираем теги в строку
#     return ','.join(tags)


# example1 = {'offer': 'Континенталь 305 / 40 / 20 Y 112 SportContact 5 5 555 112 S 40 1234567890fsfdh',
#  'width': '305',
#  'height': '40',
#  'radius': 'R20',
#  'v_ind': '112Y',
#  'brand': ['Континенталь'],
#  'line': 'SportContact 5'}

# example1 = {'offer': 'Michelin XZA 2 Energy 295 / 60 R 22 . 5 150 / 147 K Рулевая',
#  'width': '295',
#  'height': '60',
#  'radius': ['R','22','.','5'],
#  'v_ind': '150/147K',
#  'brand': ['Michelin'],
#  'line': 'XZA2 Energy'}

# example1_BIO = create_bio_tags(example1)

# print(example1['offer'],'\n')
# offer = example1['offer'].split()
# BIO_Tags = example1_BIO.split(',')

# for i in range(len(offer)):
#     print(f"{offer[i]}: {BIO_Tags[i]}")

In [13]:
x = ['R','22','.','5']
y = x.remove('R')
print(y)

None


In [14]:
l = ['5', '.', 'R', '22']

l.remove('R')
print(l)

['5', '.', '22']


In [15]:



example1 = {'offer': 'r 20 28550 michelin x-ice snow suv 116 t xl зима, MICHELIN 561981',
 'width': '285',
 'height': '50',
 'radius': 'R20',
 'v_ind': '116T',
 'brand': ['michelin', 'MICHELIN'],
 'line': 'X-Ice Snow SUV'}

example1_BIO = create_bio_tags(example1)

print(example1['offer'],'\n')
offer = example1['offer'].split()
BIO_Tags = example1_BIO.split(',')

for i in range(len(offer)):
    print(f"{offer[i]}: {BIO_Tags[i]}")

r 20 28550 michelin x-ice snow suv 116 t xl зима, MICHELIN 561981 

r: B-radius
20: I-radius
28550: O
michelin: B-brand
x-ice: B-line
snow: I-line
suv: I-line
116: B-v_ind
t: I-v_ind
xl: O
зима,: O
MICHELIN: B-brand
561981: O


In [16]:
# index = 23317
# example = dict(new_df.iloc[index])
# print(example['offer'],'\n')
# offer = example['offer'].split()
# BIO_Tags = example['BIO_Tags'].split(',')

# for i in range(len(offer)):
#     print(f"{offer[i]}: {BIO_Tags[i]}")
# example

In [17]:
new_df.to_csv('/home/sondors/Documents/price/BERT_NER/csv/Kazakov_241123/разметка/Michelin.csv', sep=';',index=False)

In [18]:
label2id = {'B-width': 1,
            'B-height': 2, 
            'B-radius': 3, 
            'I-radius': 4,
            'B-brand': 5, 
            'B-line': 6, 
            'I-line': 7,
            'B-v_ind': 8,
            'I-v_ind': 9,
            'O': 0}
    
id2label = {1: 'B-width',
            2: 'B-height', 
            3: 'B-radius', 
            4: 'I-radius',
            5: 'B-brand', 
            6: 'B-line', 
            7: 'I-line',
            8: 'B-v_ind',
            9: 'I-v_ind',
            0: 'O'}